In [38]:
from selenium import webdriver  # control browser
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  # manage chrome driver
from bs4 import BeautifulSoup
import pandas as pd
import time

In [46]:
# Amazon link
url = "https://www.amazon.eg/-/en/"

In [47]:
Product_names = []
prices = []
rates = []
def Amazon(link,user_input):
    options = Options()
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")
    options.add_argument("--disable-blink-features=AutomationControlled") 
    browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        # Get website
        browser.get(url)
        time.sleep(2)
        
        # search on products
        search_bar = browser.find_element(By.ID, "twotabsearchtextbox")
        search_bar.send_keys(user_input)
        search_bar.send_keys(Keys.RETURN)
        time.sleep(3)

        # Loop on all pages
        current_page = 1
        while current_page <= 10:
            
            print(f" Page {current_page}")
            time.sleep(5) 
            
            # scroll down in page
            for i in range(1, 5):
                browser.execute_script(f"window.scrollTo(0, {i * 1500});")
                time.sleep(1)

        
            # use BeautifulSoup
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            results = soup.find_all('div', {'data-component-type': 's-search-result'})
            print(f"Found {len(results)} products in page {current_page}.")
        
            for item in results:
                # Extract Product name
                Product_name = item.h2.text if item.h2 else "No Name"
                
                # Extract Price
                price_whole = item.find('span', {'class': 'a-price-whole'})
                price_fraction = item.find('span', {'class': 'a-price-fraction'})
                
                if price_whole:
                    full_price = f"{price_whole.text.strip()}{price_fraction.text if price_fraction else ''} EGP"
                else:
                    full_price = "Price not available"
                # Extract Rate
                rate = item.find('span',{'class': 'a-size-small a-color-base'})
                if rate:
                    rate = rate.get_text(strip=True).split(' ')[0]
                else:
                    rate = "No Rating"
                    
                Product_names.append(Product_name)
                prices.append(full_price)
                rates.append(rate)
                
            try:
                next_button = browser.find_element(By.CLASS_NAME, "s-pagination-next")
                if "s-pagination-disabled" in next_button.get_attribute("class"):
                    print("Reached the last page.")
                    break
                next_button.click()
                current_page += 1
                
            except Exception:
                print("Could not find Next button, stopping.")
                break
            
    except Exception as e:
        print(f'wrong with Amazon ==> {e}')

In [41]:
user_input = input("Enter a Product: ")
Amazon(url,user_input)

Enter a Product:  Smart Watch


 Page 1...
Found 60 products on page 1.
 Page 2...
Found 60 products on page 2.
 Page 3...
Found 60 products on page 3.
 Page 4...
Found 60 products on page 4.
 Page 5...
Found 60 products on page 5.
 Page 6...
Found 60 products on page 6.
 Page 7...
Found 30 products on page 7.
Reached the last page.


In [42]:
data = {
    'Product Name': Product_names,
    'Price': prices,
    'Rating': rates
}

df = pd.DataFrame(data)
df['Product Name'] = df['Product Name'].str.strip()
df.to_csv(r'C:\Users\HP\Desktop\amazon_results.csv', index=False, encoding='utf-8-sig')

In [48]:
data = pd.read_csv(r'C:\Users\HP\Desktop\amazon_results.csv')
data.head(5)

,Product Name,Price,Rating
0,Dl3 Mobilak Bracelet for Redmi Watch 4 / Redmi...,140.00 EGP,2.1
1,Dl3 Mobilak Bracelet Compatible with Redmi Wat...,140.00 EGP,3.9
2,Dl3 Mobilak Smart Watch Strap for Xiaomi Mi Wa...,110.00 EGP,3.7
3,Dl3 Mobilak Strap Silicone Band For Xiaomi Mi ...,75.00 EGP,3.9
4,"Redmi Watch 5 Active Smartwatch, 2.0 Inch LCD ...","1,633.53 EGP",4.4


In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  390 non-null    object
 1   Price         390 non-null    object
 2   Rating        390 non-null    object
dtypes: object(3)
memory usage: 9.3+ KB
